### Scrape all of the web data

In [1]:
import numpy as np
import pandas as pd

import rookies
import salaries
import playerstats

import re
import pickle

In [2]:
# Set URLS for various dataframes
rookies_url = 'https://www.basketball-reference.com/play-index/draft_finder.cgi?request=1&year_min=2005&year_max=2015&round_min=1&round_max=2&college_id=0&pick_overall_min=1&pick_overall_max=60&pos_is_g=Y&pos_is_gf=Y&pos_is_f=Y&pos_is_fg=Y&pos_is_fc=Y&pos_is_c=Y&pos_is_cf=Y&order_by=year_id'
salaries_url = 'http://www.espn.com/nba/salaries/_/year/{}/page/{}'

games_url = 'https://www.basketball-reference.com/leagues/NBA_{}_per_game.html'
totals_url = 'https://www.basketball-reference.com/leagues/NBA_{}_totals.html'
minutes_url = 'https://www.basketball-reference.com/leagues/NBA_{}_per_minute.html'
advanced_url = 'https://www.basketball-reference.com/leagues/NBA_{}_advanced.html'

In [5]:
# Scrape web data
rookies_df = rookies.get_rookie_data(rookies_url)
salaries_df = salaries.get_salary_data(salaries_url)

totals_df = playerstats.get_player_data(totals_url)
games_df = playerstats.get_player_data(games_url)
minutes_df = playerstats.get_player_data(minutes_url)
advanced_df = playerstats.get_player_data(advanced_url)

### Clean the salaries data frame

In [6]:
salaries_df.head()

,0,1,2,3,YEAR
0,RK,NAME,TEAM,SALARY,2005
1,1,"Shaquille O'Neal, C",Miami Heat,"$29,464,000",2005
2,2,"Dikembe Mutombo, C",Houston Rockets,"$19,684,000",2005
3,3,"Allan Houston, SG",New York Knicks,"$17,531,000",2005
4,4,"Chris Webber, C",Sacramento Kings,"$17,351,000",2005


In [7]:
# Clean salary data
salaries_df.columns = [salaries_df.iloc[0,0], salaries_df.iloc[0,1],
                       salaries_df.iloc[0,2], salaries_df.iloc[0,3], 'YEAR']

salaries_df.dropna(inplace=True)
salaries_df.drop_duplicates(inplace=True)
salaries_df.drop([0], inplace=True)
salaries_df.reset_index(inplace=True)

salaries_df.columns

Index(['index', 'RK', 'NAME', 'TEAM', 'SALARY', 'YEAR'], dtype='object')

In [8]:
salaries_df.drop('index', axis=1, inplace=True)

salaries_df.head()

,RK,NAME,TEAM,SALARY,YEAR
0,1,"Shaquille O'Neal, C",Miami Heat,"$29,464,000",2005
1,2,"Dikembe Mutombo, C",Houston Rockets,"$19,684,000",2005
2,3,"Allan Houston, SG",New York Knicks,"$17,531,000",2005
3,4,"Chris Webber, C",Sacramento Kings,"$17,351,000",2005
4,5,"Damon Stoudamire, PG",Portland Trail Blazers,"$15,750,000",2005


In [9]:
# Set Salaries to integers from currency
regex = re.compile(r'[0-9]')

salaries_df['SALARY'] = [re.findall(regex, salary) for salary in salaries_df['SALARY']]
salaries_df['SALARY'] = ["".join(salary) for salary in salaries_df['SALARY']]
salaries_df['SALARY'] = salaries_df['SALARY'].astype(int)

salaries_df.head()

,RK,NAME,TEAM,SALARY,YEAR
0,1,"Shaquille O'Neal, C",Miami Heat,29464000,2005
1,2,"Dikembe Mutombo, C",Houston Rockets,19684000,2005
2,3,"Allan Houston, SG",New York Knicks,17531000,2005
3,4,"Chris Webber, C",Sacramento Kings,17351000,2005
4,5,"Damon Stoudamire, PG",Portland Trail Blazers,15750000,2005


In [10]:
# Remove position from names column
salaries_df['NAME'] = [name.split(',') for name in salaries_df['NAME']]
salaries_df['NAME'] = [name[0] for name in salaries_df['NAME']]

salaries_df.head()

,RK,NAME,TEAM,SALARY,YEAR
0,1,Shaquille O'Neal,Miami Heat,29464000,2005
1,2,Dikembe Mutombo,Houston Rockets,19684000,2005
2,3,Allan Houston,New York Knicks,17531000,2005
3,4,Chris Webber,Sacramento Kings,17351000,2005
4,5,Damon Stoudamire,Portland Trail Blazers,15750000,2005


### Clean the rookies data frame

In [11]:
rookies_df.head()

,year_id,round,pick_overall,team_id,player,age,pos,birth_country,college
0,Year,Rd,Pk,Tm,Player,Age,Pos,Born,College
1,2015,1,1,MIN,Karl-Anthony Towns,19.227,C-F,us,University of Kentucky
2,2015,1,2,LAL,D'Angelo Russell,19.127,G,us,Ohio State University
3,2015,1,3,PHI,Jahlil Okafor,19.197,C-F,us,Duke University
4,2015,1,4,NYK,Kristaps Porzingis,19.332,F-C,lv,


In [12]:
# Clean rookies data
rookies_df.dropna(inplace=True)
rookies_df.drop_duplicates(inplace=True)
rookies_df.drop([0], axis=0, inplace=True)

rookies_df.head()

,year_id,round,pick_overall,team_id,player,age,pos,birth_country,college
1,2015,1,1,MIN,Karl-Anthony Towns,19.227,C-F,us,University of Kentucky
2,2015,1,2,LAL,D'Angelo Russell,19.127,G,us,Ohio State University
3,2015,1,3,PHI,Jahlil Okafor,19.197,C-F,us,Duke University
4,2015,1,4,NYK,Kristaps Porzingis,19.332,F-C,lv,
5,2015,1,5,ORL,Mario Hezonja,20.125,F-G,hr,


In [13]:
rookies_df.reset_index(inplace=True)
rookies_df.drop(['index'], axis=1, inplace=True)

rookies_df.head()

,year_id,round,pick_overall,team_id,player,age,pos,birth_country,college
0,2015,1,1,MIN,Karl-Anthony Towns,19.227,C-F,us,University of Kentucky
1,2015,1,2,LAL,D'Angelo Russell,19.127,G,us,Ohio State University
2,2015,1,3,PHI,Jahlil Okafor,19.197,C-F,us,Duke University
3,2015,1,4,NYK,Kristaps Porzingis,19.332,F-C,lv,
4,2015,1,5,ORL,Mario Hezonja,20.125,F-G,hr,


### Clean the totals, per game, per minute, and advanced stat data frames

In [14]:
totals_df.head()

,ast,age,blk,tov,fg,pts,fg3a,fg3,trb,orb,...,efg_pct,ft_pct,ft,fg3_pct,ranker,pf,fg2a,fg_pct,pos,year
0,AST,Age,BLK,TOV,FG,PTS,3PA,3P,TRB,ORB,...,eFG%,FT%,FT,3P%,Rk,PF,2PA,FG%,Pos,2006
1,149,29,42,108,332,887,22,5,357,106,...,.529,.784,218,.227,1,227,610,.525,PF,2006
2,4,23,0,4,4,9,5,1,5,1,...,.281,,0,.200,2,4,11,.250,SG,2006
3,20,27,16,34,121,266,1,1,140,44,...,.492,.605,23,1.000,3,92,246,.490,PF,2006
4,286,30,16,188,681,1955,653,269,332,71,...,.544,.903,324,.412,4,151,847,.454,SG,2006


In [15]:
# Clean each dataframe for totals, per game, per 36 minutes, and advanced
def clean_stats(stats):
    stats.dropna(inplace=True)
    stats.drop_duplicates(inplace=True)
    stats.drop([0], axis=0, inplace=True)
    stats.drop(['ranker'], axis=1, inplace=True)
    
    stats.reset_index(inplace=True)
    stats.drop(['index'], axis=1, inplace=True)
    
    stats['player'] = [name.replace('*', '') for name in stats['player']]
    
    return stats

In [16]:
totals = clean_stats(totals_df)
games = clean_stats(games_df)
minutes = clean_stats(minutes_df)
advanced = clean_stats(advanced_df)

In [20]:
advanced.head()

,dbpm,age,per,bpm-dum,ws,bpm,ws-dum,vorp,ast_pct,usg_pct,...,team_id,ws_per_48,ows,stl_pct,player,obpm,drb_pct,tov_pct,pos,year
0,0.4,29,17.2,,6.2,0.9,,1.4,13.2,20.1,...,SAC,.152,4.0,1.3,Shareef Abdur-Rahim,0.4,14.8,12.5,PF,2006
1,-1.9,23,-2.4,,-0.2,-12.3,,-0.1,17.9,27.2,...,DET,-0.230,-0.2,1.6,Alex Acker,-10.4,13.7,20.0,SG,2006
2,0.0,27,11.5,,1.0,-3.9,,-0.3,4.9,19.2,...,CHI,.068,0.0,1.0,Malik Allen,-3.9,15.7,11.4,PF,2006
3,-3.1,30,22.2,,9.5,3.6,,4.2,17.2,27.4,...,SEA,.150,9.8,1.8,Ray Allen,6.7,10.6,10.2,SG,2006
4,0.2,24,12.9,,1.7,-1.4,,0.2,11.2,18.8,...,BOS,.082,0.6,2.7,Tony Allen,-1.6,9.6,16.8,PG,2006


In [21]:
# Remove 'per_g' and 'per_mp' from column names
games.columns = [column.replace('_per_g','') for column in games.columns]
minutes.columns = [column.replace('_per_mp','') for column in minutes.columns]

In [31]:
# Distinguish between integers to sum and floats to average
totals_ints = ['age','ast','blk','drb','fg','fg2','fg2a','fg3','fg3a','fga','ft','fta','g','gs','mp','orb','pf','pts',
               'stl','tov','trb']
totals_floats = ['efg_pct','fg2_pct','fg3_pct','fg_pct','ft_pct']

games_ints = ['g','gs']
games_floats = ['age','ast','blk','drb','efg_pct','fg2_pct','fg2','fg2a','fg3_pct','fg3','fg3a','fg_pct','fg','fga',
                'ft_pct','ft','fta','g','gs','mp','orb','pf','pts','stl','tov','trb']

minutes_ints = ['g','gs','mp']
minutes_floats = ['age','ast','blk','drb','fg2_pct','fg2','fg2a','fg3_pct','fg3','fg3a','fg_pct','fg','fga',
                  'ft_pct','ft','fta','orb','pf','pts','stl','tov','trb']

advanced_ints = ['mp','g','dws','ws','ows']
advanced_floats = ['age','per', 'blk_pct', 'vorp', 'trb_pct','fg3a_per_fga_pct', 'obpm','ws_per_48', 'bpm','drb_pct',
                   'stl_pct','tov_pct','ts_pct','usg_pct','dbpm','fta_per_fga_pct','ast_pct','orb_pct']


In [32]:
# Set floats and integers to numeric objects
def to_num(df, ints, floats):
    nums = ints + floats
    for num in nums:
        df[num] = [pd.to_numeric(n) for n in df[num]]
    return df

In [33]:
totals = to_num(totals, totals_ints, totals_floats)
games = to_num(games, games_ints, games_floats)
minutes = to_num(minutes, minutes_ints, minutes_floats)
advanced = to_num(advanced, advanced_ints, advanced_floats)

In [34]:
advanced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8215 entries, 0 to 8214
Data columns (total 29 columns):
dbpm                8215 non-null float64
age                 8215 non-null int64
per                 8209 non-null float64
bpm-dum             8215 non-null object
ws                  8215 non-null float64
bpm                 8215 non-null float64
ws-dum              8215 non-null object
vorp                8215 non-null float64
ast_pct             8209 non-null float64
usg_pct             8209 non-null float64
orb_pct             8209 non-null float64
blk_pct             8209 non-null float64
trb_pct             8209 non-null float64
g                   8215 non-null int64
ts_pct              8168 non-null float64
dws                 8215 non-null float64
fg3a_per_fga_pct    8166 non-null float64
mp                  8215 non-null int64
fta_per_fga_pct     8166 non-null float64
team_id             8215 non-null object
ws_per_48           8209 non-null float64
ows                 

### Group each data frame by player and year

In [35]:
# Group by player and year for each frame and sum integers and average floats
def to_group(df, ints, floats):
    df_ints = df.drop(floats, axis=1)
    df_floats = df.drop(ints, axis=1)
    
    grouped_df_ints = df_ints.groupby(['player','year']).sum()
    grouped_df_floats = df_floats.groupby(['player','year']).mean()
    
    grouped_df_ints.reset_index(inplace=True)
    grouped_df_floats.reset_index(inplace=True)

    return grouped_df_ints, grouped_df_floats

In [36]:
grouped_totals_ints, grouped_totals_floats = to_group(totals, totals_ints, totals_floats)
grouped_games_ints, grouped_games_floats = to_group(games, games_ints, games_floats)
grouped_minutes_ints, grouped_minutes_floats = to_group(minutes, minutes_ints, minutes_floats)
grouped_advanced_ints, grouped_advanced_floats = to_group(advanced, advanced_ints, advanced_floats)

In [37]:
# Merge sum frames and average frames
totals = pd.merge(grouped_totals_ints, grouped_totals_floats, on=['player','year'], how='outer')
games = pd.merge(grouped_games_ints, grouped_games_floats, on=['player','year'], how='outer')
minutes = pd.merge(grouped_minutes_ints, grouped_minutes_floats, on=['player','year'], how='outer')
advanced = pd.merge(grouped_advanced_ints, grouped_advanced_floats, on=['player','year'], how='outer')

### Merge the totals, per game, and per minute data frames with the advanced statistics

In [38]:
# Merge totals frame, per games frame, and minutes, frame with advanced data
totals = pd.merge(totals, advanced, on=['player','year'], how='outer')
games = pd.merge(games, advanced, on=['player','year'], how='outer')
minutes = pd.merge(minutes, advanced, on=['player','year'], how='outer')

In [57]:
# Drop some duplicated columns and rename the old ones
for df in [totals,games,minutes]:
    df.drop(['age_y','mp_y'],axis=1,inplace=True)
    df.rename(columns={'age_x':'age','mp_x':'mp'},inplace=True)

In [58]:
minutes.drop(['g_y'], axis=1, inplace=True)
minutes.rename(columns={'g_x':'g'},inplace=True)

In [60]:
minutes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6615 entries, 0 to 6614
Data columns (total 47 columns):
player              6615 non-null object
year                6615 non-null int64
g                   6615 non-null int64
mp                  6615 non-null int64
gs                  6615 non-null int64
drb                 6611 non-null float64
age                 6615 non-null float64
stl                 6611 non-null float64
fg3                 6611 non-null float64
ft                  6611 non-null float64
fg3a                6611 non-null float64
trb                 6611 non-null float64
fg2_pct             6562 non-null float64
fta                 6611 non-null float64
ast                 6611 non-null float64
tov                 6611 non-null float64
orb                 6611 non-null float64
pts                 6611 non-null float64
fga                 6611 non-null float64
fg2                 6611 non-null float64
pf                  6611 non-null float64
fg2a                

### Merge the rookies data frame with the statistics data frames

In [63]:
# Create columns for first and fifth year in rookies frame and make name column to merge with salary frame
rookies_df['year'] = rookies_df['year_id'].astype(int) + 1
rookies_df['NAME'] = rookies_df['player']
rookies_df['YEAR'] = rookies_df['year'].astype(int) + 4

In [73]:
salaries_df.head()

,RK,NAME,TEAM,SALARY,YEAR
0,1,Shaquille O'Neal,Miami Heat,29464000,2005
1,2,Dikembe Mutombo,Houston Rockets,19684000,2005
2,3,Allan Houston,New York Knicks,17531000,2005
3,4,Chris Webber,Sacramento Kings,17351000,2005
4,5,Damon Stoudamire,Portland Trail Blazers,15750000,2005


In [65]:
# Merge each statistical frame with the rookies frame
merge_rookies_totals = pd.merge(rookies_df, totals, on=['player', 'year'], how='inner')
merge_rookies_games = pd.merge(rookies_df, games, on=['player', 'year'], how='inner')
merge_rookies_minutes = pd.merge(rookies_df, minutes, on=['player', 'year'], how='inner')

In [66]:
len(merge_rookies_totals)

480

### Merge the salaries data frames with the rookie statistics data frames

In [90]:
# Merge each rookie statistics frame with their salaries for their fifth year from the salaries frame
rookies_totals = pd.merge(merge_rookies_totals, salaries_df, on=['NAME', 'YEAR'], how='inner')
rookies_games = pd.merge(merge_rookies_games, salaries_df, on=['NAME', 'YEAR'], how='inner')
rookies_minutes = pd.merge(merge_rookies_minutes, salaries_df, on=['NAME', 'YEAR'], how='inner')

In [92]:
# Drop some duplicated columns and rename the old ones
rookies_totals.drop(['age_y'],axis=1,inplace=True)
rookies_totals.rename(columns={'age_x':'age'},inplace=True)

rookies_games.drop(['age_y','team_id_y','pos_y'],axis=1,inplace=True)
rookies_games.rename(columns={'age_x':'age','team_id_x':'team_id','pos_x':'pos'},inplace=True)

rookies_minutes.drop(['age_y'],axis=1,inplace=True)
rookies_minutes.rename(columns={'age_x':'age'},inplace=True)

In [98]:
rookies_minutes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 259 entries, 0 to 258
Data columns (total 59 columns):
draft_year          259 non-null object
round               259 non-null object
pick_overall        259 non-null object
team_id             259 non-null object
player              259 non-null object
age                 259 non-null object
pos                 259 non-null object
birth_country       259 non-null object
college             259 non-null object
first_year          259 non-null int64
g                   259 non-null int64
mp                  259 non-null int64
gs                  259 non-null int64
drb                 259 non-null float64
stl                 259 non-null float64
fg3                 259 non-null float64
ft                  259 non-null float64
fg3a                259 non-null float64
trb                 259 non-null float64
fg2_pct             258 non-null float64
fta                 259 non-null float64
ast                 259 non-null float64
tov       

### Clean up the three data frames

In [97]:
# Rename some columns to more sensible names
rookies_totals.rename(columns={'YEAR':'fifth_year','year_id':'draft_year','year':'first_year'}, inplace=True)
rookies_games.rename(columns={'YEAR':'fifth_year','year_id':'draft_year','year':'first_year'}, inplace=True)
rookies_minutes.rename(columns={'YEAR':'fifth_year','year_id':'draft_year','year':'first_year'}, inplace=True)

In [101]:
# Set some non-numerical columns to numerical
rookies_totals[['draft_year', 'round', 'pick_overall']] = rookies_totals[['draft_year','round','pick_overall']].astype(int)
rookies_games[['draft_year', 'round', 'pick_overall']] = rookies_games[['draft_year','round','pick_overall']].astype(int)
rookies_minutes[['draft_year', 'round', 'pick_overall']] = rookies_minutes[['draft_year','round','pick_overall']].astype(int)

rookies_totals['age'] = rookies_totals['age'].astype(float)
rookies_games['age'] = rookies_games['age'].astype(float)
rookies_minutes['age'] = rookies_minutes['age'].astype(float)

In [103]:
# Uppercase all of the column names for consistency
rookies_totals.columns = [column.upper() for column in rookies_totals.columns]
rookies_games.columns = [column.upper() for column in rookies_games.columns]
rookies_minutes.columns = [column.upper() for column in rookies_minutes.columns]

In [108]:
# Drop player who is missing lots of data
rookies_totals.drop(100,inplace=True)
rookies_games.drop(100,inplace=True)
rookies_minutes.drop(100,inplace=True)

In [127]:
# Fill in NaN values in the three point percantage column
rookies_totals[rookies_totals['FG3_PCT'].isna()] = 0
rookies_games[rookies_games['FG3_PCT'].isna()] = 0
rookies_games[rookies_games['FG3_PCT'].isna()] = 0

In [118]:
rookies_totals.reset_index(inplace=True)
rookies_games.reset_index(inplace=True)
rookies_minutes.reset_index(inplace=True)

In [120]:
rookies_totals.drop(['index'],axis=1,inplace=True)
rookies_games.drop(['index'],axis=1,inplace=True)
rookies_minutes.drop(['index'],axis=1,inplace=True)

In [122]:
# Drop some useless columns
rookies_totals.drop(['NAME','RK'],axis=1,inplace=True)
rookies_games.drop(['NAME','RK'],axis=1,inplace=True)
rookies_minutes.drop(['NAME','RK'],axis=1,inplace=True)

In [132]:
rookies_totals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258 entries, 0 to 257
Data columns (total 58 columns):
DRAFT_YEAR          258 non-null int64
ROUND               258 non-null int64
PICK_OVERALL        258 non-null int64
TEAM_ID             258 non-null object
PLAYER              258 non-null object
AGE                 258 non-null float64
POS                 258 non-null object
BIRTH_COUNTRY       258 non-null object
COLLEGE             258 non-null object
FIRST_YEAR          258 non-null int64
AST                 258 non-null int64
BLK                 258 non-null int64
TOV                 258 non-null int64
FG                  258 non-null int64
PTS                 258 non-null int64
FG3A                258 non-null int64
FG3                 258 non-null int64
TRB                 258 non-null int64
ORB                 258 non-null int64
FTA                 258 non-null int64
G                   258 non-null int64
FGA                 258 non-null int64
FG2                 258 non-nu

In [381]:
# Send these three datasets to pickles
with open('rookies_totals.pickle', 'wb') as to_write:
    pickle.dump(rookies_totals, to_write)
    
with open('rookies_games.pickle', 'wb') as to_write:
    pickle.dump(rookies_games, to_write)

with open('rookies_minutes.pickle', 'wb') as to_write:
    pickle.dump(rookies_minutes, to_write)